In [1]:
!pip install gradio sentence-transformers faiss-cpu gTTS SpeechRecognition soundfile pyttsx3

In [2]:

# -----------------------------

# 🍽️ Golden Fork Bistro Chatbot

# -----------------------------

import gradio as gr

import time, datetime, tempfile

import numpy as np

from gtts import gTTS

import speech_recognition as sr

from sentence_transformers import SentenceTransformer

import faiss

import os

In [3]:

# -----------------------------

# 📝 Bistro Knowledge Base

# -----------------------------

documents = [

    "Welcome to The Golden Fork Bistro – a premier dining destination at 123 Gourmet Lane, Foodie City, offering a culinary journey of authentic European and American flavors. You can reach us at +9876543210. Our ambiance is perfect for romantic dinners and special events.",

    "Our bistro is open to serve you Tuesday through Sunday, from 12 PM (noon) to 10 PM. Please note that we are closed on Mondays.",

    "Reservations are highly recommended, especially on weekends and for large parties. You can secure a table by calling us directly at +9876543210 or by visiting our website: www.goldenforkbistro.com.",

    "Signature Dishes: Pan-Seared Salmon, Wagyu Beef Burger, Classic Carbonara.",

    "Vegan Options: Vegan Mushroom Risotto, Roasted Cauliflower Steak.",

    "Appetizers: Bruschetta, Truffle Fries, Crisp Calamari.",

    "Desserts: Chocolate Lava Cake, Crème brûlée, Tiramisu."

]

In [4]:

# -----------------------------

# 🔍 Embedding + FAISS

# -----------------------------

model = SentenceTransformer("all-MiniLM-L6-v2")

document_embeddings = np.array(model.encode(documents)).astype("float32")

dimension = document_embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)

index.add(document_embeddings)



def retrieve_relevant_documents(query, top_k=2):

    query_embedding = np.array(model.encode([query])).astype("float32")

    distances, indices = index.search(query_embedding, top_k)

    return [documents[i] for i in indices[0]]

In [5]:

# -----------------------------

# 🤖 Response Generator

# -----------------------------

def generate_response(query):

    context = retrieve_relevant_documents(query)

    if "hour" in query.lower():

        return "⏰ We're open Tue-Sun, 12 PM - 10 PM. Closed Mondays."

    if "dish" in query.lower():

        return "🍝 Popular dishes: Pan-Seared Salmon, Wagyu Burger, Carbonara."

    if "reservation" in query.lower():

        return "📞 Call +9876543210 or book online at goldenforkbistro.com."

    return f"👨‍🍳 Here’s what I found: {' '.join(context)}"


In [6]:

# -----------------------------

# 🗣️ Speech-to-Text

# -----------------------------

def speech_to_text(audio):

    recognizer = sr.Recognizer()

    try:

        with sr.AudioFile(audio) as source:

            audio_data = recognizer.record(source)

            return recognizer.recognize_google(audio_data)

    except Exception:

        return "Sorry, I couldn't understand. Please try again."

In [7]:

# -----------------------------

# 🔊 Text-to-Speech

# -----------------------------

def text_to_speech(text):

    try:

        tts = gTTS(text=text, lang="en")

        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")

        tts.save(temp_file.name)

        return temp_file.name

    except Exception as e:

        print("TTS error:", e)

        return None


In [8]:

# -----------------------------

# 💬 Typing Effect

# -----------------------------

def typing_effect(text, delay=0.05):

    response = ""

    for char in text:

        response += char

        time.sleep(delay)

        yield response

In [9]:

from datetime import datetime

import pytz



# -----------------------------

# 📩 Chat Logic (with IST timezone)

# -----------------------------

tts_enabled = {"status": True}



def respond(message, chat_history):

    # ✅ Get current time in Hyderabad (IST)

    ist = pytz.timezone("Asia/Kolkata")

    timestamp = datetime.now(ist).strftime("%I:%M %p")  # e.g., 09:50 PM



    # Add user message

    chat_history.append({"role": "user", "content": f"{message}\n\n🕒 {timestamp}"})

    yield "", chat_history, None  # reset audio



    # Generate bot response

    bot_message = generate_response(message)



    # Start TTS

    audio_path = None

    if tts_enabled["status"]:

        audio_path = text_to_speech(bot_message)



    # Typing effect

    partial = ""

    for part in typing_effect(bot_message):

        partial = part

        if chat_history and chat_history[-1]["role"] == "assistant":

            chat_history[-1] = {"role": "assistant", "content": f"{partial}\n\n🕒 {timestamp}"}

        else:

            chat_history.append({"role": "assistant", "content": f"{partial}\n\n🕒 {timestamp}"})

        yield "", chat_history, None



    yield "", chat_history, audio_path

In [ ]:

# -----------------------------

# 🎨 CSS

# -----------------------------

css_style = """

.light {

    --bg-color: #ffffffd9;

    --text-color: #2c2c2c;

    --user-msg-bg: #ffe0b2;

    --bot-msg-bg: #e1f5fe;

}

.dark {

    --bg-color: #2e2e2e;

    --text-color: #f5f5f5;

    --user-msg-bg: #8d6e63;

    --bot-msg-bg: #4fc3f7;

}

.gradio-container {

    background: linear-gradient(-45deg, #fceabb, #f8b500, #e8cbc0, #636fa4);

    background-size: 400% 400%;

    animation: gradientBG 15s ease infinite;

    font-family: 'Playfair Display', serif;

    color: var(--text-color);

}

.message.user { background: var(--user-msg-bg); }

.message.assistant { background: var(--bot-msg-bg); }



@keyframes gradientBG {

    0% { background-position: 0% 50%; }

    50% { background-position: 100% 50%; }

    100% { background-position: 0% 50%; }

}



/* 🔹 Fix button alignment */

.equal-btn button {

    width: 100% !important;

    height: 50px;

    font-size: 15px;

    font-weight: bold;

}

"""

# -----------------------------

# 🚀 Gradio UI

# -----------------------------

with gr.Blocks(css=css_style, theme=gr.themes.Soft()) as demo:

    gr.HTML("<h1 style='text-align:center'>🍽️ The Golden Fork Bistro 🍴</h1>")

    gr.HTML("<p style='text-align: center; font-style: italic;'>Your personal dining assistant ✨</p>")



    with gr.Column():

        chatbot = gr.Chatbot(

            label="Chat with us",


            height=400,

            avatar_images=(

                "https://cdn-icons-png.flaticon.com/512/3135/3135715.png",

                "https://cdn-icons-png.flaticon.com/512/4712/4712071.png"

            )

        )



        # Quick buttons row - symmetric

        with gr.Row(equal_height=True):

            with gr.Column(scale=1, elem_classes="equal-btn"):

                quick_q1 = gr.Button("⏰ Hours?")

            with gr.Column(scale=1, elem_classes="equal-btn"):

                quick_q2 = gr.Button("🍴 Popular dishes?")

            with gr.Column(scale=1, elem_classes="equal-btn"):

                quick_q3 = gr.Button("📞 Reservations?")



        # Input + Send/Clear row - symmetric

        with gr.Row(equal_height=True):

            msg = gr.Textbox(placeholder="Type your question...", scale=4)

            with gr.Column(scale=1, elem_classes="equal-btn"):

                send_btn = gr.Button("Send")

            with gr.Column(scale=1, elem_classes="equal-btn"):

                clear_btn = gr.Button("Clear", variant="secondary")



        # Voice input/output row - symmetric

        with gr.Row(equal_height=True):

            voice_input = gr.Audio(sources=["microphone"], type="filepath", label="🎙️ Speak")

            voice_output = gr.Audio(label="🔊 Bot Voice")



    # Bind logic

    send_btn.click(respond, [msg, chatbot], [msg, chatbot, voice_output])

    msg.submit(respond, [msg, chatbot], [msg, chatbot, voice_output])

    clear_btn.click(lambda: (None, [], None), None, [msg, chatbot, voice_output])



    quick_q1.click(lambda: "What are your hours?", None, msg)

    quick_q2.click(lambda: "What are your popular dishes?", None, msg)

    quick_q3.click(lambda: "How can I make a reservation?", None, msg)



    voice_input.change(fn=speech_to_text, inputs=voice_input, outputs=msg)





if __name__ == "__main__":

    demo.launch(debug=True)



C:\Users\Yeshwanth\AppData\Local\Temp\ipykernel_2872\3326414790.py:87: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme, css. Please pass these parameters to launch() instead.
  with gr.Blocks(css=css_style, theme=gr.themes.Soft()) as demo:


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\Yeshwanth\anaconda3\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\Yeshwanth\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\Yeshwanth\anaconda3\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\Yeshwanth\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdo